Model Transparency/ Explainability

Compute shapley values with SHAP'S DeepExplainer class

In [ ]:
import shap

Here we are using 100 training samples (distrib_samples) and validating on 30 samples (num_explanations) starting from 3000 index (start). Assume model has three inputs, two unstructured texts and one structured text

In [ ]:
shap.initjs()
import keras
# select a set of samples to take an expectation over
distrib_samples=[unstructured_text1_train[:100],unstructured_text2_train[:100],structured_train[:100]]
session=keras.backend.tensorflow_backend.get_session()

explainer=shap.DeepExplainer(model, distrib_samples, session)
num_explanations=30
start=3000

shap_values=explainer.shap_values([unstructured_text1_valid[start:start+num_explanations],unstructured_text2_valid[start:start+num_explanations],structured_valid[start:start+num_explanations]])

Generalized model transparency using shap.summary_plot

In [ ]:
num2word_text1={}
num2word_text2={}
num2word_structured={}

### get all words
structured_fields=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14']
word_index_text1= tokenizer_text1.word_index
word_index_text2= tokenizer_text2.word_index
for w in word_index_text1.keys():
  num2word_text1=[word_index_text1[w]]=w
for w in word_index_text2.keys():
  num2word_text2=[word_index_text2[w]]=w

### summary plot for each of the inputs
shap.summary_plot(np,.concatenate([shap_values[0][2]],axis=1), feature_names=structured_fields, max_display=15)
shap.summary_plot(np,.concatenate([shap_values[0][0]],axis=1), feature_names= list(num2word_text1.values()), max_display=15)
shap.summary_plot(np,.concatenate([shap_values[0][1]],axis=1), feature_names= list(num2word_text2.values()), max_display=15)

In [ ]:
### here we will find shap.force_plot for each test output
### find shap.force_plot for 2nd valid data
i=2
test_inp=[unstructured_text1_valid[start+i].reshape(1,-1),unstructured_text2_valid[start+i].reshape(1,-1),structured_valid[start+i].reshape(1,-1)] 
pred=model.predict(test_inp)
print("model outputs: {}".format(pred))
x_test_words1=np.asarray(list(map(lambda x: num2word_text1.get(x,"None"),test_inp[0][0].tolist())))
x_test_words2=np.asarray(list(map(lambda x: num2word_text2.get(x,"None"),test_inp[1][0].tolist())))

x_test_words=[]
for j in range(len(x_test_words1)):
  x_test_words.append(x_test_words1[j])
for j in range(len(x_test_words2)):
  x_test_words.append(x_test_words2[j])
x_test_words=np.asarray(x_test_words)

Get positive features which improves model performance and also negative features that reduces model performance

In [ ]:
import nltk
shap_user=np.concatenate([shap_values[0][0],shap_values[0][1]], axis=1)[i]
shap_user_importance=np.argsort(shap_user)

## Want to see top 10 users
top_user_n=10

### Find most negative influence words
neg_cols=[x_test_words[shap_user_importance[c]] for c in range(top_user_n)]
### Get all neg_ind for words except pronouns, prepositions
indexes_neg=[neg_cols.index(x) for x in set(neg_cols) if (nltk.pos_tag(nltk.word_tokenize(x))[0][1] !='PRP' and nltk.pos_tag(nltk.word_tokenize(x))[0][1] !='IN')]
neg_cols=[neg_cols[i] for i in indexes_neg]
neg_vals=[shap_user[shap_user_importance[c]] for c in range(top_user_n)]
neg_vals=[neg_vals[i] for i in indexes_neg]
neg_indexes=[shap_user_importance[c] for c in range(top_user_n)]
neg_indexes=[neg_indexes[i] for i in index_neg]

### Find most positive influence words
pos_cols=[x_test_words[shap_user_importance[-(c+1)]] for c in range(top_user_n)]
### Get all neg_ind for words except pronouns, prepositions
indexes_pos=[pos_cols.index(x) for x in set(pos_cols) if (nltk.pos_tag(nltk.word_tokenize(x))[0][1] !='PRP' and nltk.pos_tag(nltk.word_tokenize(x))[0][1] !='IN')]
pos_cols=[pos_cols[i] for i in indexes_pos]
pos_vals=[shap_user[shap_user_importance[c]] for c in range(top_user_n)]
pos_vals=[pos_vals[i] for i in indexes_pos]
pos_indexes=[shap_user_importance[-(c+1)] for c in range(top_user_n)]
pos_indexes=[pos_indexes[i] for i in index_pos]

print('positive features', neg_cols,neg_vals,neg_indexes)
print('negative features', pos_cols,pos_vals,pos_indexes)
main_feats=neg_indexes[:]
main_feats.extend(pos_indexes[:])
main_col_names=[x_test_words[k] for k in main_feats]
shap.force_plot(explainer.expected_value[0], shap_user[main_feats], feature_names=main_col_names, text_rotation=90, figsize=(60,60))

Get the snippets (past 5 words and next 5 words) for each positive & negative model features

In [ ]:
snippets_neg=[]
for n1 in range(len(neg_cols)):
  l1=np.where(x_test_words1 == neg_cols[n1])[0]
  if np.shape(l1)[0]>0:
    l2=l1[0]
    snippets_neg.append(' '.join(x_test_words1[l2-5:l2+5]))

snippets_pos=[]
for n1 in range(len(pos_cols)):
  l1=np.where(x_test_words1 == pos_cols[n1])[0]
  if np.shape(l1)[0]>0:
    l2=l1[0]
    snippets_pos.append(' '.join(x_test_words1[l2-5:l2+5]))